In [71]:
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import os
import pandas as pd
import numpy as np
import numpy as np

def otsu_thresholding(importancia: np.ndarray) -> float:
    hist, _ = np.histogram(importancia, bins=400, range=(importancia[-1], importancia[0]))  # Calcular el histograma
    rango = (importancia[0] - importancia[-1])/400
    hist = hist / np.sum(hist)  # Normalizar el histograma para obtener probabilidades
    max_sigma = 0
    threshold = 0

    for t in range(1, len(hist)):
        w0 = np.sum(hist[:t])  # Probabilidad acumulada para la clase 0
        w1 = 1 - w0  # Probabilidad acumulada para la clase 1
        if w0 == 0 or w1 == 0:
            continue

        mu0 = np.sum(np.arange(t) * hist[:t]) / w0  # Media ponderada para la clase 0
        mu1 = np.sum(np.arange(t, len(hist)) * hist[t:]) / w1  # Media ponderada para la clase 1

        sigma = w0 * w1 * ((mu0 - mu1) ** 2)  # Calcular varianza entre clases

        if sigma > max_sigma:
            max_sigma = sigma
            threshold = t
    return threshold * rango


def leerArchivos(carpeta:str) -> list[str]:
    cuerpo = []
    archivos_txt = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.txt')]
    for archivo in archivos_txt:
        ruta_completa = os.path.join(carpeta, archivo)
        with open(ruta_completa, 'r',encoding="utf-8") as file:
            contenido = file.read()
            cuerpo.append(contenido)
    return cuerpo

def normalizarTexto(texto:str) -> (str,list[str],list[str]):
    stopwords_es = set(stopwords.words('spanish'))
    signos_puntuacion = string.punctuation + '¡¿'
    # Eliminar signos de puntuación
    oraciones = texto.split(".")[:-1]
    oraciones_normalizadas = []
    oraciones_stop = []
    texto_normalizado = ""
    for oracion in oraciones:
        texto_sin_puntuacion = ''.join([caracter for caracter in oracion if caracter not in signos_puntuacion])
        # Tokenizar el texto en palabras
        palabras = word_tokenize(texto_sin_puntuacion, language='spanish')
        palabras = [palabra.lower() for palabra in palabras]
        # Eliminar stopwords
        palabras_sin_stopwords = [palabra for palabra in palabras if palabra not in stopwords_es]
        oraciones_stop.append(' '.join(palabras))
        oraciones_normalizadas.append(' '.join(palabras_sin_stopwords))
        texto_normalizado = texto_normalizado+ ' '.join(palabras_sin_stopwords)
    return texto_normalizado, oraciones_normalizadas,oraciones_stop

def normalizarCuerpo(cuerpo:list[str])->list[str]:
    cuerpo_normalizado = []
    oraciones_normalizadas = []
    oraciones_stop = []
    for documento in cuerpo:
        c_norm,oraciones_cuerpo,oraciones_cuerpo_stop = normalizarTexto(documento)
        cuerpo_normalizado.append(c_norm)
        oraciones_normalizadas.append(oraciones_cuerpo)
        oraciones_stop.append(oraciones_cuerpo_stop)
    return cuerpo_normalizado,oraciones_normalizadas,oraciones_stop

def calcularITFDTCuerpo(cuerpo:list[str])->pd.DataFrame:
    vectorizador_tfidf = TfidfVectorizer()
    matriz_tfidf = vectorizador_tfidf.fit_transform(cuerpo)
    palabras = vectorizador_tfidf.get_feature_names_out()
    dataframes = []
    for doc_idx, documento in enumerate(cuerpo):
        data = []
        palabras_doc = documento.split()
        palabras_doc = [palabra for palabra in palabras if palabra in palabras_doc]
        for palabra in palabras_doc:
            index_palabra = vectorizador_tfidf.vocabulary_.get(palabra)
            valor_idf = vectorizador_tfidf.idf_[index_palabra]
            valor_tfidf = matriz_tfidf[:, index_palabra].toarray().flatten()[doc_idx]
            valor_tf = valor_tfidf/valor_idf
            data.append([
                palabra,
                doc_idx + 1,
                valor_tf,
                valor_idf,
                valor_tfidf
            ])
        dataframes.append(pd.DataFrame(data, columns=["palabra", "documento", "TF", "IDF", "TFIDF"]))
    return dataframes

def obtenerMejoresPuestos(dfdocumento:pd.DataFrame,columna:str,cantidad:int=10,tipo:str="cantidad")->pd.DataFrame:
    if(tipo == "cantidad"):
        return dfdocumento.sort_values(by=columna, ascending=False).head(cantidad)
    else:
        cuerpo_ordenado = dfdocumento.sort_values(by=columna, ascending=False)
        t = otsu_thresholding(cuerpo_ordenado[columna].to_numpy())
        return cuerpo_ordenado[cuerpo_ordenado[columna] >= t]

def similitudOracionesEmbeddings(cuerpo_oracion):
    cuerpo_oracion_ranked = []
    orden_og = []
    for oraciones in cuerpo_oracion:
        word_embeddings_model = Word2Vec(oraciones, vector_size=100, window=5, min_count=1, sg=0)
        # Calcular los embeddings de las oraciones
        sentence_embeddings = []
        for oracion in oraciones:
            palabras = oracion.split()
            embedding_oracion = np.mean([word_embeddings_model.wv[palabra] for palabra in palabras if palabra in word_embeddings_model.wv] or [np.zeros(100)], axis=0)
            sentence_embeddings.append(embedding_oracion)
        # Calcula la similitud entre oraciones
        similarity_matrix = np.zeros([len(oraciones), len(oraciones)])
        for i in range(len(oraciones)):
            for j in range(len(oraciones)):
                if(i==j):
                    similarity_matrix[i][j]=0
                elif((np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[j])) == 0):
                    similarity_matrix[i][j] = 0
                else:
                    similarity_matrix[i][j] = np.dot(sentence_embeddings[i], sentence_embeddings[j]) / (np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[j]))
        ranking_oraciones = np.argsort(np.sum(similarity_matrix, axis=1))[::-1]
        cuerpo_oracion_ranked.append([oraciones[idx] for idx in ranking_oraciones])
        orden_og.append(ranking_oraciones)
    return cuerpo_oracion_ranked,orden_og


def generarResumenDocumento(df_documento:pd.DataFrame,oraciones_cuerpo_ranked:list[str],orden_oraciones:list[int],oraciones_og:list[str],size:int=5) -> list[str]:
    cantidad_oraciones = len(oraciones_cuerpo_ranked)
    mejores_puestos = obtenerMejoresPuestos(df_documento,"TFIDF",tipo="otsu")
    palabras = mejores_puestos["palabra"].to_numpy()
    oraciones_tfidf = np.zeros((cantidad_oraciones))
    for idx,oracion in enumerate(oraciones_cuerpo_ranked):
        palabras_oracion = oracion.split()
        for palabra in palabras_oracion:
            if palabra in palabras:
                oraciones_tfidf[idx] +=1
    oraciones_redundates = oraciones_tfidf[:cantidad_oraciones*2//3]
    oraciones_extra = oraciones_tfidf[cantidad_oraciones*2//3:]
    oraciones_redundates_sort =np.argsort(oraciones_redundates)[::-1]
    oraciones_extra_sort = np.argsort(oraciones_extra)[::-1]
    resumen = []
    for i in range(size-1):
        indice_oracion =oraciones_redundates_sort[i]
        indice_oracion_inicial = orden_oraciones[indice_oracion]
        resumen.append(oraciones_og[indice_oracion_inicial])
    for i in range(1):
        indice_oracion =oraciones_extra_sort[i]+cantidad_oraciones*2//3
        indice_oracion_inicial = orden_oraciones[indice_oracion]
        resumen.append(oraciones_og[indice_oracion_inicial])
    return resumen

def generarResumenCuerpo(resumenesDocumentos,longitud=5):
    oracionesImportantes = []
    for documento in resumenesDocumentos:
        print(documento)
        oracionesImportantes.append(documento[0])
        oracionesImportantes.append(documento[1])
    word_embeddings_model = Word2Vec(oracionesImportantes, vector_size=100, window=5, min_count=1, sg=0)
    sentence_embeddings = []
    for oracion in oracionesImportantes:
        palabras = oracion.split()
        embedding_oracion = np.mean([word_embeddings_model.wv[palabra] for palabra in palabras if palabra in word_embeddings_model.wv] or [np.zeros(100)], axis=0)
        sentence_embeddings.append(embedding_oracion)
        # Calcula la similitud entre oraciones
    similarity_matrix = np.zeros([len(oracionesImportantes), len(oracionesImportantes)])
    for i in range(len(oracionesImportantes)):
        for j in range(len(oracionesImportantes)):
            if(i==j):
                similarity_matrix[i][j]=0
            elif((np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[j])) == 0):
                similarity_matrix[i][j] = 0
            else:
                similarity_matrix[i][j] = np.dot(sentence_embeddings[i], sentence_embeddings[j]) / (np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[j]))
    ranking_oraciones = np.argsort(np.sum(similarity_matrix, axis=1))[::-1][:longitud]
    return [oracionesImportantes[idx] for idx in ranking_oraciones]

In [72]:
cuerpo = leerArchivos("documentos")
cuerpo,oraciones_cuerpo,oraciones_stop = normalizarCuerpo(cuerpo)
df_cuerpo = calcularITFDTCuerpo(cuerpo)
oraciones_cuerpo_ranked,orden_oraciones = similitudOracionesEmbeddings(oraciones_cuerpo)
resumenesIndividuales = []
for idx in range(len(cuerpo)):
    resumenesIndividuales.append(generarResumenDocumento(df_cuerpo[idx],oraciones_cuerpo_ranked[idx],orden_oraciones[idx],oraciones_stop[idx]))
generarResumenCuerpo(resumenesIndividuales)

['por el contrario gracias al aprendizaje automático muchos de los dispositivos que verás en el futuro obtendrán experiencia y conocimientos a partir de la forma en que son utilizados para poder ofrecer una experiencia al usuario personalizada', 'por ejemplo los filtros de spam de correo electrónico utilizan este tipo de aprendizaje con el fin de detectar qué mensajes son correo basura y separarlos de aquellos que no lo son', 'éste es un sencillo ejemplo de cómo los algoritmos pueden usarse para aprender patrones y utilizar el conocimiento adquirido para tomar decisiones', 'los primeros ordenadores personales que estuvieron disponibles para los consumidores a partir de la década de 1980 fueron programados explícitamente para realizar ciertas acciones', 'aprendizaje automático el aprendizaje automático en inglés machine learning es uno de los enfoques principales de la inteligencia artificial']


['por ejemplo los filtros de spam de correo electrónico utilizan este tipo de aprendizaje con el fin de detectar qué mensajes son correo basura y separarlos de aquellos que no lo son',
 'por el contrario gracias al aprendizaje automático muchos de los dispositivos que verás en el futuro obtendrán experiencia y conocimientos a partir de la forma en que son utilizados para poder ofrecer una experiencia al usuario personalizada']